In [3]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# pytorch relates imports
import torch
import torch.nn as nn
import torch.optim as optim

## DATA WRANGLING BOSTON

In [6]:
boston = load_boston()
X, y = boston.data, boston.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

X_train = torch.tensor(X_train).float()
y_train = torch.tensor(y_train).view(-1, 1).float()

X_test = torch.tensor(X_test).float()
y_test = torch.tensor(y_test).view(-1, 1).float()

datasets = torch.utils.data.TensorDataset(X_train, y_train)
train_iter = torch.utils.data.DataLoader(datasets, batch_size=10, shuffle=True)

In [7]:
batch_size = 50
num_epochs = 200
learning_rate = 0.0001
size_hidden1 = 100
size_hidden2 = 50
size_hidden3 = 10
size_hidden4 = 1

In [12]:
def NN(sz = 10):
    features = nn.Sequential(
        torch.nn.Linear(13, sz),
        torch.nn.Linear(sz, sz),
        torch.nn.Linear(sz, sz),
        torch.nn.Linear(sz, sz),
        torch.nn.Linear(sz, sz),
        #extension starts here
        torch.nn.Linear(sz, 1)
        
    )
    return(features)

In [35]:
model = NN(sz=100)
criterion = nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=0.0000001)

In [36]:
def train(model_inp, num_epochs = num_epochs):
    #optimizer = torch.optim.RMSprop(model_inp.parameters(), lr=learning_rate)
    for epoch in range(num_epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        for inputs, labels in train_iter:
            # forward pass
            outputs = model_inp(inputs)
            # defining loss
            loss = criterion(outputs, labels)
            # zero the parameter gradients
            optimizer.zero_grad()
            # computing gradients
            loss.backward()
            # accumulating running loss
            running_loss += loss.item()
            # updated weights based on computed gradients
            optimizer.step()
        if epoch % 20 == 0:    
            print('Epoch [%d]/[%d] running accumulative loss across all batches: %.3f' %
                  (epoch + 1, num_epochs, running_loss))
        running_loss = 0.0

In [37]:
train(model, 1000)

Epoch [1]/[1000] running accumulative loss across all batches: 80212.423
Epoch [21]/[1000] running accumulative loss across all batches: 28447.797
Epoch [41]/[1000] running accumulative loss across all batches: 27650.197
Epoch [61]/[1000] running accumulative loss across all batches: 26992.547
Epoch [81]/[1000] running accumulative loss across all batches: 26370.776
Epoch [101]/[1000] running accumulative loss across all batches: 26183.364
Epoch [121]/[1000] running accumulative loss across all batches: 25727.673
Epoch [141]/[1000] running accumulative loss across all batches: 25453.420
Epoch [161]/[1000] running accumulative loss across all batches: 25165.400
Epoch [181]/[1000] running accumulative loss across all batches: 25105.261
Epoch [201]/[1000] running accumulative loss across all batches: 24720.629
Epoch [221]/[1000] running accumulative loss across all batches: 24553.718
Epoch [241]/[1000] running accumulative loss across all batches: 24440.409
Epoch [261]/[1000] running accu

In [31]:
# helper function to get accuracy
def get_accuracy(output, targets):
    """Helper function to print the accuracy"""
    predictions = output.argmax(dim=1, keepdim=True).view_as(targets)
    return predictions.eq(targets).float().mean().item()



acc = []

max_len = len(train_iter)
for batch_idx, (x, y) in enumerate(train_iter):
        output = model(x)
        print(output)
        print(y)
        accuracy = get_accuracy(output, y.float())
        if batch_idx % 10 == 0:
            print(
                "Batch {}/{} \t".format(batch_idx, max_len) + 
                "Accuracy %.0f" % (accuracy * 100) + "%"
            )
        acc.append(accuracy)
    
avg_acc = np.mean(acc)
print('overall test accuracy on Boston: {:.02f} %'.format(avg_acc * 100))

tensor([[30.6294],
        [23.8833],
        [33.0699],
        [32.2612],
        [31.9739],
        [26.3832],
        [25.9582],
        [19.9050],
        [21.6845],
        [24.3534]], grad_fn=<AddmmBackward>)
tensor([[50.0000],
        [19.1000],
        [34.9000],
        [36.5000],
        [27.9000],
        [22.7000],
        [16.2000],
        [13.3000],
        [ 7.2000],
        [23.4000]])
Batch 0/36 	Accuracy 0%
tensor([[30.7545],
        [23.4478],
        [25.6390],
        [27.9437],
        [23.2662],
        [19.1250],
        [14.2092],
        [22.0389],
        [28.6641],
        [25.1982]], grad_fn=<AddmmBackward>)
tensor([[29.1000],
        [21.7000],
        [15.0000],
        [26.6000],
        [15.6000],
        [12.7000],
        [13.2000],
        [14.9000],
        [22.0000],
        [22.0000]])
tensor([[24.0204],
        [17.4724],
        [18.8079],
        [25.1820],
        [20.4192],
        [30.9359],
        [24.6936],
        [30.9253],
        [2